In [15]:
# Uncomment the "!pip install" line below to use on Google Colab. Run this cell and restart the runtime kernel.
# https://colab.research.google.com/github/spedas/pyspedas/blob/mth5/pyspedas/mth5/examples/comparison_with_published_results.ipynb
# !pip install git+https://github.com/spedas/pyspedas.git@mth5
# !pip install git+https://github.com/kujaku11/mth5.git

# Results from Heyns et al. (2020)

This notebook show example of the measurements from the Fredericksbur (FRD) station and nearby stations obtained from FDSN network.
Figure 1 by [Heyns et al. (2020)]((https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2020SW002557)) illustrates such measurement in comparion with GIC measurments 

Heyns, M. J., Lotz, S. I., & Gaunt, C. T. (2021). Geomagnetic pulsations driving geomagnetically induced currents. Space Weather, 19, e2020SW002557. https://doi.org/10.1029/2020SW002557

In [ ]:
from IPython.display import Image
Image(url= "https://agupubs.onlinelibrary.wiley.com/cms/asset/015c5548-201a-4fd0-a676-b8b872089875/swe21080-fig-0001-m.jpg", width=900, height=600)

# Plot FRD station data using PySPEDAS GMAG

To obtain the measurements from FRD station we use `themis.gmag` module, which provides access to the [large set of magnetometer data](https://themis.ssl.berkeley.edu/gmag/gmag_list.php?selyear=4000&selmonth=13&smap=on&sinfo=on&saelist=on&ae=on).
[FRD station](https://www.usgs.gov/programs/geomagnetism/science/fredericksburg-frd) is a gold standard in Space Physics. It is a part of [USGS Geomagnetic Observatory](https://www.usgs.gov/programs/geomagnetism/science/observatories). 

In [ ]:
import pyspedas, pytplot

date_start = '2015-06-22T01:45:00'
date_end = '2015-06-22T02:20:00'
# Load data from FDR station
pyspedas.themis.gmag(sites='frd', trange=[date_start, date_end], time_clip=True)

# Split data into individual components
pytplot.split_vec('thg_mag_frd')

# Plot y component
pytplot.tplot('thg_mag_frd_y')

The output from pytplot is the same as in Figure 1 from Heyns et al. (2020) 

# Search for nearby stations

FRD station coordinates are presented in Table 1 from Heyns et al. (2020)

| Station         | Geographic latitude | Geographic longitude | Geomagnetic MLat. | Geomagnetic MLon. |
|-----------------|---------------------|----------------------|-------------------|-------------------|
| Fredericksburg  | 38.2°               | −77.4°               | 47.8°             | −0.2°             |

To find nearby station we can use the [https://ds.iris.edu/gmap/](https://ds.iris.edu/gmap/) webpage.

In [ ]:
from IPython.display import IFrame
from datetime import datetime, timedelta

date_fmt = "%Y-%m-%dT%H:%M:%S"
sdate = datetime.strptime(date_start, date_fmt).strftime('%Y-%m-%d') # Only date 
edate =(datetime.strptime(date_start, date_fmt) + timedelta(days=1)).strftime('%Y-%m-%d') # Only date but plus one day
net = '*' # Select all networks
lat = 38.2 # Geographic latitude
lon = -77.4 # Geographic longitude
rad = 8  # Radius of the area

# Construct url using fstring
url= f'https://ds.iris.edu/gmap/#network={net}&starttime={sdate}&endtime={edate}&latitude={lat}&longitude={lon}&maxradius={rad}&drawingmode=radial&planet=earth'
print(url)
IFrame(url, 900,500)

# Limit stations of interest

The previous request provided multiple stations, including FRD. Now we need to limit the search to the stations that can provide magnetometer data in IRIS. According to the Standard for the Exchange of Earthquake Data (or **SEED**) [reference manual appendix A](https://www.fdsn.org/pdf/SEEDManual_V2.4_Appendix-A.pdf), the SEED format uses three letters to name the channels (Band code, Instrument code and Orientation code). We are interested in magnetomenter data which has an instrument code **F**. However, the wildcard '\*F\*' will return channels that also ends with 'F' and do not represent magnetic field data. Hence, we include only list of channels that correspond to low frequency magnetic field measurements: MF\*, LF\*, VF\*, UF\*. Note, that we did not include the orientation code, which can be **Z**, **N**, **E** (Vertical, North-South, East-West). To exclude channels that may represent something else (e.g.,  **VFP** Packet Buffer Usage), we can further limit our search to East-West oriented magnetomenter: MFE, LFE, VFE, UFE

In [ ]:
channel = "MFE,LFE,VFE,UFE" 
url= f'https://ds.iris.edu/gmap/#network={net}&channel={channel}&starttime={sdate}&endtime={edate}&latitude={lat}&longitude={lon}&maxradius={rad}&drawingmode=radial&planet=earth'
print(url)
IFrame(url, 900,650)

# Plot station data using PySPEDAS MTH5 

Let select station  **REU49** from network **4P** and load data using mth5 format.

To obtain the station data we use `load_fdsn` function of `mth5` module. Not that we use `nodownload` flag, which prevents recurrent download of the data. Once the data is obtained for selected period the .h5 cache file is created. `nodownload=False` will force to create the .h5 files every time.

In [ ]:
from pyspedas.mth5.load_fdsn import load_fdsn

load_fdsn(network="4P", station="REU49", trange=[date_start, date_end], nodownload=True)

# Split data into individual components
pytplot.split_vec('fdsn_4P_REU49')

# Plot all data and the y component
pytplot.tplot(['fdsn_4P_REU49', 'fdsn_4P_REU49_y'])

Let explore another station **GAW50** from the same network **4P**

In [ ]:
load_fdsn(network="4P", station="GAW50", trange=[date_start, date_end], nodownload=True)

# Split data into individual components
pytplot.split_vec('fdsn_4P_GAW50')

# Plot all data and the y component
pytplot.tplot(['fdsn_4P_GAW50', 'fdsn_4P_GAW50_y'])

# Compare the results

Finally, we can compare the results by plotting de-trended By from all 3 stations. For simply subtract the median value.

In [ ]:
import numpy as np

tplot_vars = ['thg_mag_frd_y', 'fdsn_4P_REU49_y', 'fdsn_4P_GAW50_y']

for var in tplot_vars:
   time, by = pytplot.get_data(var)  # Unpack tplot variable
   by -= np.median(by)  # Subtract median   
   pytplot.options(var, opt_dict={"legend_names": [var], "ytitle": var.split("_")[2].upper()}) #  Customize labels

pytplot.tplot(tplot_vars)

# Handling data from other stations

There could be a scenario when the data from other stations is inconsistent. For example, as of November 2023 the data from the station **REQ48** of the same network **4P** shows significant outlier and potentially flipped. Let's explore this station and plot the results. 


In [ ]:
load_fdsn(network="4P", station="REQ48", trange=[date_start, date_end], nodownload=True)
pytplot.split_vec('fdsn_4P_REQ48')
pytplot.options('fdsn_4P_REQ48_y', 'yrange', [-2, 2])  # Manually set the range, because 4P REQ48 have outliers in the signal
pytplot.tplot(['fdsn_4P_REQ48', 'fdsn_4P_REQ48_y', 'thg_mag_frd_y']) # Add FRD station ius added for comparison 

Another scenario is when the IRDS map indicate availability, however the data is unavailable. 
The next example tries to load all stations. If the data is not available the MTH processing raises and exception which can be handed by try-except block. However, you can use flag `noexeptions` to bypass this.

In [ ]:
stations = ['ALW49', 'GAW50', 'REP47', 'REQ48', 'REU49', 'TNU48', 'TNV48']
tplot_str = []
for sta in stations:
   load_fdsn(network="4P", station=sta, trange=[date_start, date_end], nodownload=True, noexception=True)   
   pytplot.split_vec(f'fdsn_4P_{sta}')
   var = f'fdsn_4P_{sta}_y'
   tplot_str.append(var)
   pytplot.options(var, opt_dict={"legend_names": [var], "ytitle": var.split("_")[2].upper()}) #  Customize labels

pytplot.tplot(tplot_str)

In the previous example, the data from station **TNU48** has not been created. Next example shows how processing of the absent data can be handled in your code

In [ ]:
def tplot_fdsn_station(network, station, trange):
   """Plot data only is load_fdsn does not return an error. Print error details"""

   # tplot variable string
   var = None
   
   try:
      load_fdsn(network=network, station=station, trange=trange)
      var = f'fdsn_{network}_{station}'
   except Exception as E:
      print("FDSN loading error: ", type(E), E)
      
   if var:
      pytplot.tplot(var)
   else:
      # Nothing to plot
      pass

tplot_fdsn_station("4P", "TNU48", [date_start, date_end])

Next example simulate the situation when FDSN cannot find or load the data. This will intentionally set not existing station

In [ ]:
tplot_fdsn_station("4P", "TNU0", [date_start, date_end])